In [1]:
import torch
from torch import hub

In [2]:
resnet18_model = hub.load("pytorch/vision:master", "resnet18", pretrained=True)

Using cache found in /Users/sanchitnevgi/.cache/torch/hub/pytorch_vision_master
Downloading: "https://download.pytorch.org/models/resnet18-5c106cde.pth" to /Users/sanchitnevgi/.cache/torch/hub/checkpoints/resnet18-5c106cde.pth


1. PyTorch defaults to an immediate execution model (eager mode). The underlying instructions are immediately run on C++
2. *TorchScript* is used to serialize a model into independent functions (for production use)
3. *ONNX* is a standard vendor-agnostic format for model specification

# Chapter 3: Tensors

1. PyTorch tensors or NumPy arrays, on the other hand, are views over (typically) **contiguous memory blocks** containing **unboxed** C numeric types rather than Python objects
2. Each element is a 32-bit, 4 byte float
3. Indexing
    1. `tensor[1:2,5:6]`
4. Named Tensors
5. By default, tensors are `float32` or `int64`
6. A `Tensor` is a view of a `torch.Storage` instance. Multiple tensors can index the same storage even if they index into the data differently.
7. Using `Storage` makes some operations like *transpose*, *slicing* inexpensive as there is no memory re-allocation.
8. Changing the subtensor will have a side effect on the original tensor
9. Contiguous memory is more efficient (data locality)

In [2]:
tensor = torch.rand(4, 5)
tensor

tensor([[0.4631, 0.9883, 0.8731, 0.0668, 0.9128],
        [0.9470, 0.3323, 0.7959, 0.9149, 0.6834],
        [0.3909, 0.6917, 0.5042, 0.1998, 0.4877],
        [0.0207, 0.7719, 0.8862, 0.6709, 0.0439]])

In [3]:
# Size of a tensor
tensor.size()

torch.Size([4, 5])

In [4]:
# Tensor Indexing
tensor[1:-1,1:-1]

tensor([[0.3323, 0.7959, 0.9149],
        [0.6917, 0.5042, 0.1998]])

In [5]:
# Adding a new dimension to the tensor. Same as tensor.unsqueeze()
tensor[None]

tensor([[[0.4631, 0.9883, 0.8731, 0.0668, 0.9128],
         [0.9470, 0.3323, 0.7959, 0.9149, 0.6834],
         [0.3909, 0.6917, 0.5042, 0.1998, 0.4877],
         [0.0207, 0.7719, 0.8862, 0.6709, 0.0439]]])

## Named Tensors

In [10]:
# Named tensors
img_t = torch.randn(2, 4, 4)
img_t_named = img_t.refine_names(..., "channels", "width", "height")

img_t

tensor([[[ 1.5427,  1.3242,  1.2242,  0.3213],
         [ 0.5507, -0.9032, -0.7180, -1.8293],
         [-1.1005,  2.0100, -2.0273, -1.9066],
         [ 0.7365, -0.5707, -1.4878,  0.0032]],

        [[-0.6709, -0.9744,  1.7123, -1.1352],
         [-0.7160, -2.4199,  0.2918,  0.2005],
         [-0.0837,  0.2023, -0.4316, -0.3629],
         [-0.3429,  1.8585,  1.4647, -0.5072]]])

In [11]:
# Another way of defining named tensors
img_r = torch.randn(2, 3, 4, names=["channels", "width", "height"])
img_r

tensor([[[-0.9128,  0.7514, -0.2208,  1.4657],
         [-0.5490,  0.0058, -0.4195,  0.6694],
         [ 0.9004,  0.9597,  0.0488,  1.7796]],

        [[-0.5128,  1.5673, -1.3727,  0.2322],
         [ 0.5986, -1.2913, -1.6881,  0.2693],
         [-0.1088, -1.2981, -0.1335, -0.4401]]],
       names=('channels', 'width', 'height'))

In [12]:
# Align a tensor to the names of another
weights_named = torch.randn(2, names=["channels"])
print("Unaligned: ", weights_named.size())

weights_aligned = weights_named.align_as(img_r)
print("Aligned: ", weights_aligned.size())

Unaligned:  torch.Size([2])
Aligned:  torch.Size([2, 1, 1])


In [13]:
# Any function with dimension argument, takes the name as well
img_r.sum("channels").size()

torch.Size([3, 4])

In [14]:
# Reset names
img_r.rename(None)

tensor([[[-0.9128,  0.7514, -0.2208,  1.4657],
         [-0.5490,  0.0058, -0.4195,  0.6694],
         [ 0.9004,  0.9597,  0.0488,  1.7796]],

        [[-0.5128,  1.5673, -1.3727,  0.2322],
         [ 0.5986, -1.2913, -1.6881,  0.2693],
         [-0.1088, -1.2981, -0.1335, -0.4401]]])

## Tensor data types

In [16]:
# Integer tensors by default are int64
int_tensor = torch.tensor([2, 2])
int_tensor.dtype

torch.int64

In [17]:
# Float tensors are float32 by default
float_tensor = torch.tensor([1.0, 1.0])
float_tensor.dtype

torch.float32

In [25]:
# Setting dtype on initialization
ones = torch.ones(5, 4, dtype=torch.short)
ones

tensor([[1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1],
        [1, 1, 1, 1]], dtype=torch.int16)

In [27]:
# Casting to dtype
ones.double()
ones.to(torch.double)

tensor([[1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.],
        [1., 1., 1., 1.]], dtype=torch.float64)

## Tensor API
Refer the book

## Tensor View

In [28]:
# A storage is always 1 dimensional
torch.Storage([1, 2, 3])

 1.0
 2.0
 3.0
[torch.FloatStorage of size 3]

In [30]:
# We can access the underlying storage
tensor = torch.tensor([1, 2, 3])
tensor.storage()

 1
 2
 3
[torch.LongStorage of size 3]

In [31]:
# Changing the storage, changes all the tensors viewing it
tensor.storage()[0] = 4
tensor

tensor([4, 2, 3])

In [32]:
# Some tensors have special methods with trailing underscores (zero_). 
# This means that they perform in-place operation. Otherwise they return a new tensor
tensor.zero_()
tensor

tensor([0, 0, 0])

In [41]:
# A tensor has metadata - size, offset, and stride; to properly index into the storage
tensor = torch.randint(0, 5, size=(3, 3))
print(tensor)
print("stride: ", tensor.stride())
print("offset: ", tensor.storage_offset())

tensor([[1, 4, 0],
        [4, 4, 0],
        [1, 3, 1]])
stride:  (3, 1)
offset:  0


In [43]:
row = tensor[1]
print(row.stride(), row.storage_offset())

(1,) 3


In [44]:
# Changing a sub-tensor will modify the original tensor
row[0] = 10
tensor

tensor([[ 1,  4,  0],
        [10,  4,  0],
        [ 1,  3,  1]])

In [45]:
# To avoid this, clone the tensor
row_clone = tensor[1].clone()
row_clone

tensor([10,  4,  0])

In [47]:
# Transpose
a = torch.randint(0, 4, size=(2, 3))
print(a)
print(a.t())

tensor([[1, 0, 1],
        [1, 3, 2]])
tensor([[1, 1],
        [0, 3],
        [1, 2]])


In [50]:
# Explicitly make the tensor contiguous
print(a.t().is_contiguous())
b = a.t().contiguous()
a.is_contiguous()

False


True

In [61]:
a = torch.tensor(list(range(9)))
b = a.view(3, 3)
print("Same storage:", id(a.storage()) == id(b.storage()))
print(a.stride(), b.stride())

Same storage: True
(1,) (3, 1)


## Moving tensors to the GPU

In [53]:
# Use the device argument to specify a device
# gpu_tensor = torch.ones(10, 5, device="cuda")

# Move to the gpu
# gpu_tensor = torch.ones(10, 5).to(device="cuda")

# Chapter 4: Real-world data representation using tensors